In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from vamos import optimize, OptimizeConfig, NSGAIIConfig, ZDT1
from vamos.tuning import ParamSpace, RandomSearchTuner, TuningTask
from vamos.tuning.space import Real, Int, Categorical
from vamos.foundation.metrics import compute_hypervolume
from vamos.foundation.problem.problem import make_problem_selection

plt.style.use("ggplot")
print("Tuning tools loaded!")

## 1. Define a Custom Parameter Space

Create a search space with different parameter types:

In [ ]:
# Define a richer parameter space for NSGA-II
custom_space = ParamSpace(
    params={
        "pop_size": Int("pop_size", 60, 150),
        "crossover_prob": Real("crossover_prob", 0.7, 0.95),
        "crossover_eta": Real("crossover_eta", 10.0, 30.0),
        "mutation_prob": Real("mutation_prob", 0.1, 0.3, log=True),
        "mutation_eta": Real("mutation_eta", 10.0, 30.0),
        "selection_pressure": Categorical("selection_pressure", [2, 3, 4]),
    }
)

# Sample a few configurations to see what they look like
rng = np.random.default_rng(42)
for i in range(3):
    sample = custom_space.sample(rng)
    print(f"Sample {i + 1}: {sample}")

## 2. Simple Random Search Tuning

Use `RandomSearchTuner` to find good hyperparameters for NSGA-II on ZDT1:

In [ ]:
def tuning_evaluator(config, ctx):
    """Evaluation function called by the tuner."""

    # Convert config into AlgorithmConfig
    algo_config = (
        NSGAIIConfig()
        .pop_size(config["pop_size"])
        .crossover("sbx", prob=config["crossover_prob"], eta=config["crossover_eta"])
        .mutation("pm", prob=config["mutation_prob"], eta=config["mutation_eta"])
        .engine("numpy")
        .fixed()
    )

    # Create problem selection
    selection = make_problem_selection("zdt1", n_var=10)
    problem = selection.instantiate()

    # Reference point for hypervolume (slightly worse than nadir)
    ref_point = np.array([1.1, 1.1])

    # Run optimization with budget similar to comparison
    opt_config = OptimizeConfig(
        problem=problem,
        algorithm="nsgaii",
        algorithm_config=algo_config,
        termination=("n_eval", ctx.budget),
        seed=ctx.seed,
    )
    result = optimize(opt_config)

    # Compute hypervolume
    hv = compute_hypervolume(result.F, ref_point)
    return hv


# Create tuning task
task = TuningTask(
    name="nsgaii_zdt1_tuning",
    evaluator=tuning_evaluator,
    space=custom_space,
    budget=3000,  # evals per run
    n_trials=20,  # total configs to test
    seed=42,
)

tuner = RandomSearchTuner(task)
results = tuner.run()

best_trial = results.get_best_trial()
print(f"\nBest Hypervolume: {best_trial.score:.6f}")
print("Best Configuration:")
for k, v in best_trial.config.items():
    print(f"  {k}: {v}")

## 3. Next Steps: Advanced Programmatic Tuning

For more advanced tuning capabilities, including **Parallel Execution**, **Raceband Strategies**, and the new **RacingTuner API**, please refer to:

### [17_programmatic_tuning.ipynb](./17_programmatic_tuning.ipynb)

That notebook demonstrates how to:
- Use the `RacingTuner` for efficient budget allocation.
- Tune multiple algorithms simultaneously.
- Visualize tuning progress dynamically.